In [3]:
import sqlite3
from sqlite3 import Error
from sqlite3 import IntegrityError
from ipyfilechooser import FileChooser
import pandas as pd
import io
import requests

In [4]:
starting_directory = '/Users/Joseph'
chooser = FileChooser(starting_directory)
display(chooser)

FileChooser(path='/Users/Joseph', filename='', title='', show_hidden=False, select_desc='Select', change_desc=…

In [6]:
file=chooser.selected
folder_path=chooser.selected_path
print(file)
print(folder_path)

/Users/Joseph/Desktop/GRC_Data/tap_baseline_output.csv
/Users/Joseph/Desktop/GRC_Data


In [7]:
baseline_output = pd.read_csv(file, index_col=0)
print(baseline_output)

           Time   n  Number  Instantaneous Speed  Interval Speed   Bias  Tap  \
0         0.006  68       0               0.0000          0.0000  0.000    0   
1         0.058  72       0               0.0000          0.0000  0.000    0   
2         0.120  78       0               0.0000          0.0000  0.000    0   
3         0.162  81       0               0.0000          0.0000  0.000    0   
4         0.226  81       0               0.0000          0.0000  0.000    0   
...         ...  ..     ...                  ...             ...    ...  ...   
80052  1199.893  52      28               0.1992          0.1135  0.545    0   
80053  1199.950  51      28               0.1832          0.1068  0.000    0   
80054  1199.991  51      28               0.0000          0.0000  0.000    0   
80055  1200.034  51      28               0.0000          0.0000  0.000    0   
80056  1200.075  51      28               0.0000          0.0000  0.000    0   

       Puff        x        y  ...     

In [8]:
start = 350 #<----- set the start time of the range
point = 360 #<------- set the end time of your range
chunkeddata0 = baseline_output[baseline_output["Time"] >= start]
chunkeddata = chunkeddata0[chunkeddata0["Time"] <= point]
print(chunkeddata)

          Time    n  Number  Instantaneous Speed  Interval Speed   Bias  Tap  \
3969   350.055  157      83               0.1114          0.0899  0.654    0   
3970   350.202  156      82               0.1180          0.0942  0.688    0   
3971   350.314  156      82               0.1324          0.1029  0.663    0   
3972   350.396  156      82               0.1265          0.0939  0.637    0   
3973   350.476  155      82               0.1332          0.0998  0.625    0   
...        ...  ...     ...                  ...             ...    ...  ...   
66298  359.680   84      47               0.0911          0.1051  0.435    0   
66299  359.760   84      47               0.0886          0.1044  0.391    0   
66300  359.843   84      47               0.0920          0.1116  0.391    0   
66301  359.918   84      47               0.0805          0.0908  0.391    0   
66302  359.993   84      47               0.0803          0.0796  0.391    0   

       Puff        x        y  ...     

In [9]:
chunkeddata.to_csv('/Users/Joseph/Desktop/NRSC510B/sample_baseline_output.csv')

In [10]:
# tap_url = 'https://osf.io/du9bj/files/osfstorage/650a2f9f1e76a4230e8a99a5?raw=true'
tap_url='https://github.com/MyYummyPancake/NRSC510B/blob/main/tap_output.csv?raw=true'
# s=requests.get(tap_url).content
# tap_output=pd.read_csv(io.StringIO(s.decode('utf-8')))
tap_output=pd.read_csv(tap_url, on_bad_lines='skip', index_col=0)
print(tap_output)

         time  dura   dist      prob     speed  plate  taps         dataset  \
0     599.959  3.21  0.581  0.803922  0.180997      1   1.0  F32D8.13_gk584   
1     609.960  2.54  0.509  0.648148  0.200394      1   2.0  F32D8.13_gk584   
2     619.914  1.84  0.409  0.750000  0.222283      1   3.0  F32D8.13_gk584   
3     629.897  1.73  0.364  0.474576  0.210405      1   4.0  F32D8.13_gk584   
4     639.989  1.21  0.247  0.536232  0.204132      1   5.0  F32D8.13_gk584   
..        ...   ...    ...       ...       ...    ...   ...             ...   
334   859.979  0.79  0.163  0.450000  0.206329     11  27.0   vps-35_ok1880   
335   869.983  1.04  0.203  0.350877  0.195192     11  28.0   vps-35_ok1880   
336   879.979  0.82  0.163  0.456140  0.198780     11  29.0   vps-35_ok1880   
337   889.911  0.72  0.133  0.415094  0.184722     11  30.0   vps-35_ok1880   
338  1189.993  3.23  0.687  0.583333  0.212693     11  31.0   vps-35_ok1880   

         Gene  Allele  
0    F32D8.13   gk584  
1  

In [18]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


In [ ]:
# Function for basic data validation
def validate_data(df):
    # Define expected data types
    expected_dtypes = {
        'Unnamed: 0': 'int64',
        'time': 'float64',
        'dura': 'float64',
        'dist': 'float64',
        'prob': 'float64',
        'speed': 'float64',
        'plate': 'int64',
        'taps': 'float64',
        'dataset': 'object',
        'Gene': 'object',
        'Allele': 'object'
    }
    
    # Type validation
    for col, expected_dtype in expected_dtypes.items():
        if df[col].dtype != expected_dtype:
            return False, f"Column {col} has incorrect data type {df[col].dtype}. Expected {expected_dtype}."
    
    # Range validation for specific columns
    if df['prob'].min() < 0 or df['prob'].max() > 1:
        return False, "Column 'prob' has values outside the range [0, 1]."
    
    return True, "Data is valid."

In [ ]:
is_valid, validation_message = validate_data(df)
if is_valid:
    # Create SQLite database and insert data
    try:
        conn = sqlite3.connect('c_elegans_data.db')
        df.to_sql('experiment_data', conn, if_exists='replace', index=False)
        etl_status = "Data successfully inserted into SQLite database."
    except IntegrityError as e:
        etl_status = f"Data insertion failed: {e}"
else:
    etl_status = f"Data validation failed: {validation_message}"

print(etl_status)

In [11]:
conn=sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_tap_data.db')
tap_output.to_sql('tap_response_data', conn, if_exists='replace', index=False)
baseline_output.to_sql('tap_baseline_data', conn, if_exists='replace', index=False)

13290439

In [12]:
conn.close()